### Задача 1

In [2]:
import spacy
import numpy as np
nlp = spacy.load('en_core_web_sm')
word_1 = 'cat'
word_2 = 'cow'

/Users/nadezdaoskina/.pyenv/versions/3.10.9/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.7.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
vector_1 = nlp(word_1).vector
vector_2 = nlp(word_2).vector

In [4]:
cosine_similarity = np.dot(vector_1, vector_2) /(np.linalg.norm(vector_1) * np.linalg.norm(vector_2))

In [5]:
result = round(cosine_similarity, 3)
print(result)

0.775


### Задача 2

In [6]:
sentence_1 = "Сегодня очень холодно на улице"
sentence_2 = "На улице сегодня очень холодно"

In [11]:
vector_1 = nlp(sentence_1).vector
vector_2 = nlp(sentence_2).vector

# Расчёт косинусной меры между векторами
cosine_similarity = np.dot(vector_1, vector_2) / (np.linalg.norm(vector_1) * np.linalg.norm(vector_2))

# Округление результата до 3 знаков после запятой
result = round(cosine_similarity, 3)

# Вывод результата
print(f"Косинусная мера между предложениями: {result}")

Косинусная мера между предложениями: 0.9070000052452087


In [ ]:
# Предлагаем свериться с возможным вариантом решения.

import spacy
import numpy as np
nlp = spacy.load("ru_core_news_lg")
def cosine_similarity(s1, s2):
    doc1 = nlp(s1)
    doc2 = nlp(s2)
    vec1 = doc1.vector
    vec2 = doc2.vector
    if np.linalg.norm(vec1)*np.linalg.norm(vec2) > 0:
      return np.dot(vec1, vec2) / (np.linalg.norm(vec1)*np.linalg.norm(vec2))
    else:
      return 0
s1 = "Сегодня очень холодно на улице"
s2 = "На улице сегодня очень холодно"
similarity = cosine_similarity(s1, s2)
print(f"Косинусная мера угла между предложениями '{s1}' и '{s2}': {similarity}")
s1 = "Обезьяны любят бананы"
s2 = "Собаки не любят молоко"
similarity = cosine_similarity(s1, s2)
print(f"Косинусная мера угла между предложениями '{s1}' и '{s2}': {similarity}")


### Задача 3

In [12]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
data = pd.read_csv("data/product_description.csv")

# Load spaCy language model
nlp = spacy.load("en_core_web_sm")

# Preprocessing function
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Apply preprocessing to the description column
data['processed_text'] = data['description'].apply(preprocess_text)

# Vectorize the processed text using TF-IDF
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data['processed_text'])

# Calculate cosine similarity matrix
cosine_sim_matrix = cosine_similarity(vectors)

# Find top-5 most similar products
similarities = []
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        similarities.append((cosine_sim_matrix[i, j], data['product_name'][i], data['product_name'][j]))

# Sort by similarity score in descending order
similarities = sorted(similarities, key=lambda x: x[0], reverse=True)

# Display top-5 most similar product pairs
top_5_similar = similarities[:5]
for score, product1, product2 in top_5_similar:
    print(f"Products: {product1} and {product2} - Similarity: {round(score, 3)}")


Products: Chair and Table - Similarity: 0.259
Products: Ottoman and Dresser - Similarity: 0.251
Products: Chair and Sofa - Similarity: 0.251
Products: Chair and Bed - Similarity: 0.246
Products: Table and Dresser - Similarity: 0.241


In [15]:
# Предлагаем свериться возможным вариантом решения

import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Загрузка модели языка
nlp = spacy.load("en_core_web_sm")
# Загрузка датасета с описанием товаров
data = pd.read_csv('data/product_description.csv')
# Предобработка текста
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)
data['processed_text'] = data['description'].apply(preprocess_text)
# Создание матрицы векторов для каждого товара
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data['processed_text'])
# Расчет косинусной меры угла между каждой парой товаров
cosine_similarities = cosine_similarity(vectors)
# Отображение топ-5 товаров, которые наиболее похожи друг на друга
for i, row in data.iterrows():
    similar_indices = cosine_similarities[i].argsort()[:-6:-1]
    similar_items = [(cosine_similarities[i][j], data['product_name'][j]) for j in similar_indices if j != i]
    print(f"Top 5 similar items for {row['product_name']}:\n{similar_items}\n")

Top 5 similar items for Chair:
[(0.2593876295324481, 'Table'), (0.25055127750774986, 'Sofa'), (0.24561802198055008, 'Bed'), (0.11469978576166628, 'Desk')]

Top 5 similar items for Sofa:
[(0.25055127750774986, 'Chair'), (0.16258619842261424, 'Ottoman'), (0.13009805981223888, 'Bookshelf'), (0.08915638255306323, 'Dresser')]

Top 5 similar items for Table:
[(0.2593876295324481, 'Chair'), (0.24096224354397955, 'Dresser'), (0.16801123015453076, 'Ottoman'), (0.08941424217490086, 'Bed')]

Top 5 similar items for Bed:
[(0.24561802198055008, 'Chair'), (0.08941424217490086, 'Table'), (0.0880478167497522, 'TV Stand'), (0.024477160600450878, 'Dresser')]

Top 5 similar items for Bookshelf:
[(0.21924673021588997, 'TV Stand'), (0.15880746901055803, 'Cabinet'), (0.13009805981223888, 'Sofa'), (0.10482488731104447, 'Dresser')]

Top 5 similar items for Desk:
[(0.22556771102764483, 'Dresser'), (0.11469978576166628, 'Chair'), (0.09609143719920325, 'Cabinet'), (0.02234343682389054, 'Ottoman')]

Top 5 similar